In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics
from utils.preserves import load_object
from utils.watchmen import DirectLimitWatchman, PcaLimitWatchman, IsoForestWatchman, LinearPredictWatchman

In [4]:
SEED = 585

# Preparing datasets

In [5]:
datasets = {
    0: GhlKasperskyDataset(),
    1: TepHarvardDataset(),
    2: TepKasperskyDataset(),
    3: SwatItrustDataset(),
}

In [6]:
shake_kwargs = {
    'random_state': SEED,
    'valid2test_ratio': 0.3,
}
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)

# Preparing watchhouse

In [7]:
n = 5
watchhouse = {
    0: {w: None for w in range(n)},
    1: {w: None for w in range(n)},
    2: {w: None for w in range(n)},
    3: {w: None for w in range(n)},
}

In [8]:
for d in datasets:
    for w in watchhouse[d]:
        watchhouse[d][w] = load_object(f'{datasets[d].__class__.__name__}-watchman_{w}')

# Examine

## Throw stones

In [9]:
stones = dict()
for d in datasets:
    stones[d] = dict()
    for w in watchhouse[d]:
        stones[d][w] = list()

In [ ]:
for d in datasets:
    for data, faults, info in tqdm(datasets[d].valid_generator(), desc=f'Detect on validation {d}'):
        for w in watchhouse[d]:
            detect = watchhouse[d][w].predict(data)
            stones[d][w].append(pd.concat([faults, detect], axis=1))
            stones[d][w][-1].index.name = info

Detect on validation 0:   0%|          | 0/14 [00:00<?, ?it/s]

Detect on validation 1:   0%|          | 0/3150 [00:00<?, ?it/s]

## Individual results

In [ ]:
metrics = ('precision', 'recall', 'f1_score')
results = {str(datasets[d]): pd.DataFrame(columns=metrics) for d in datasets}

In [ ]:
for d in datasets:
    for w in tqdm(watchhouse[d], desc=f'Collect stones d{d}'):
        exam_paper = pd.DataFrame(columns=metrics)
        for i_st, st in enumerate(stones[d][w]):
            exam_paper.loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], st.iloc[:, 1:])
        results[str(datasets[d])].loc[str(watchhouse[d][w]), metrics] = exam_paper.mean().values

## Ensembling results

In [ ]:
thresholds = (2, 4)
for threshold in thresholds:
    for d in datasets:
        exam_paper = pd.DataFrame(columns=metrics)
        for i_st, st in enumerate(tqdm(stones[d][0], desc=f'Collect stones d{d}')):
            opinions = pd.concat([stones[d][w][i_st].iloc[:, 1:].sum(axis=1) for w in watchhouse[d]], axis=1)
            detect = (opinions.sum(axis=1) >= threshold).astype('uint8')
            exam_paper.loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], detect)
        results[str(datasets[d])].loc[f'ensemble(threshold={threshold})', metrics] = exam_paper.mean().values

In [ ]:
for d in results:
    print(d)
    display(results[d].sort_values(by='f1_score', ascending=False))